* The "Spline" Condition (Smoothness): The key is that these piecewise polynomials are joined smoothly. At each internal "knot" $x_i$ (where two pieces meet), the spline ensures that:
    * The values are equal: $S_{i-1}(x_i) = S_i(x_i) = y_i$. (The curves connect).
    * The first derivatives are equal: $S'_{i-1}(x_i) = S'_i(x_i)$. (The slope is the same).
    * The second derivatives are equal: $S''_{i-1}(x_i) = S''_i(x_i)$. (The curvature is the same).

In [14]:
import numpy as np
import pandas as pd
import math
from typing import Tuple

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

# Linear Spline

## Algorithm

**Input:** $n+1$ data points $(x_0, y_0), (x_1, y_1), \dots, (x_n, y_n)$, sorted by $x$.

**Output:** A table of coefficients $(a_{i,0}, a_{i,1})$ for each piecewise linear function $S_i(x) = a_{i,1} \cdot x + a_{i,0}$, valid on the interval $[x_i, x_{i+1}]$.

* Constructing the Spline Segments

For each interval $i = 0, 1, \dots, n-1$:

1.  **Calculate the Slope (Degree 1 coefficient):**
    $a_{i,1} = \frac{y_{i+1} - y_i}{x_{i+1} - x_i}$

2.  **Calculate the Constant (Degree 0 coefficient):**
    $a_{i,0} = y_i - a_{i,1} \cdot x_i$

3.  **Store Coefficients:**
    Save the pair $(a_{i,0}, a_{i,1})$ for the interval $[x_i, x_{i+1}]$.

The final spline $S(x)$ is the collection of all $S_i(x)$ functions:
$S(x) = \begin{cases} 
      a_{0,1} \cdot x + a_{0,0} & x \in [x_0, x_1] \\
      a_{1,1} \cdot x + a_{1,0} & x \in [x_1, x_2] \\
      \vdots & \vdots \\
      a_{n-1,1} \cdot x + a_{n-1,0} & x \in [x_{n-1}, x_n] 
   \end{cases}$

* Evaluating the Spline at $x_{val}$

To find the interpolated value at a specific point $x_{val}$:

1.  **Find the Interval:** Determine which interval $[x_i, x_{i+1}]$ contains $x_{val}$. (i.e., find $i$ such that $x_i \le x_{val} \le x_{i+1}$).
2.  **Select Coefficients:** Retrieve the coefficients $a_{i,0}$ and $a_{i,1}$ for that segment $i$.
3.  **Calculate:** Compute the value using the standard form:
    $y_{val} = S_i(x_{val}) = a_{i,1} \cdot x_{val} + a_{i,0}$

In [15]:
def linear_spline_interpolation(points):
    """
    Calculates the set of linear spline functions S_i(x) for each interval
    in standard polynomial form: S_i(x) = a_1*x + a_0.
    
    Args:
        points (list of tuples): List of (x, y) data points, sorted by x.

    Returns:
        pd.DataFrame: A DataFrame containing the coefficients for each
                      linear segment S_i(x).
    """
    n = len(points) - 1
    segments_data = []

    for i in range(n):
        x_i, y_i = points[i]
        x_ip1, y_ip1 = points[i+1]
        
        h = x_ip1 - x_i
        if h <= 0:
            raise ValueError(f"Points are not sorted or x values are not distinct at index {i}.")
            
        # S_i(x) = a_1 * x + a_0
        # a_1 = m_i = (y_{i+1} - y_i) / (x_{i+1} - x_i)
        a_1 = (y_ip1 - y_i) / h
        
        # a_0 = y_i - m_i * x_i
        a_0 = y_i - a_1 * x_i
        
        segments_data.append({
            'i': i,
            'Interval': f"[{x_i}, {x_ip1}]",
            'a_0 (Degree 0)': a_0,
            'a_1 (Degree 1)': a_1,
        })
        
    segments_df = pd.DataFrame(segments_data, columns=['i', 'Interval', 'a_0 (Degree 0)', 'a_1 (Degree 1)'])
    return segments_df


## Results

In [16]:
# 1. Define the data points from the image
points = [
    (1.4, 2.4347052),
    (1.5, 2.5473627),
    (1.6, 2.6495552),
    (1.7, 2.7412610),
    (1.8, 2.8225627),
    (1.9, 2.8936474),
    (2.0, 2.9548000),
    (2.1, 3.0064203),
    (2.2, 3.0489801)
]

In [17]:
# 2. Generate the spline segments
segments_table = linear_spline_interpolation(points)

print("--- Linear Spline Segments ---")
segments_table.style

--- Linear Spline Segments ---


,i,Interval,a_0 (Degree 0),a_1 (Degree 1)
0,0,"[1.4, 1.5]",0.857500,1.126575
1,1,"[1.5, 1.6]",1.014475,1.021925
2,2,"[1.6, 1.7]",1.182262,0.917058
3,3,"[1.7, 1.8]",1.359132,0.813017
4,4,"[1.8, 1.9]",1.543038,0.710847
5,5,"[1.9, 2.0]",1.731748,0.611526
6,6,"[2.0, 2.1]",1.922394,0.516203
7,7,"[2.1, 2.2]",2.112664,0.425598


## Further Calculation

In [18]:
def evaluate_linear_spline(segments_df, x_val):
    """
    Evaluates the piecewise linear spline at a specific x value.
    This function is now designed to work with the table of a_0 and a_1 coefficients.
    
    Args:
        segments_df (pd.DataFrame): The DataFrame of spline segments from
                                    linear_spline_interpolation.
        x_val (float): The x-value to interpolate at.

    Returns:
        float: The interpolated y-value.
    """
    
    # 1. Find the correct segment
    segment_found = None
    last_x_ip1 = -np.inf
    
    for _, row in segments_df.iterrows():
        interval_str = row['Interval']
        # Parse interval string "[x_i, x_ip1]"
        x_i, x_ip1 = [float(x) for x in interval_str.strip('[]').split(', ')]
        last_x_ip1 = x_ip1
        
        if x_i <= x_val <= x_ip1:
            segment_found = row
            break
            
    # Handle edge case: x_val is slightly larger than the last node
    # (e.g., due to floating point), or exactly on the last node.
    if segment_found is None and np.isclose(x_val, last_x_ip1):
         segment_found = segments_df.iloc[-1]
    
    if segment_found is None:
        # Check if x_val is outside the total range
        first_x_i = float(segments_df.iloc[0]['Interval'].strip('[]').split(', ')[0])
        if x_val < first_x_i or x_val > last_x_ip1:
            raise ValueError(f"x_val = {x_val} is outside the interpolation range [{first_x_i}, {last_x_ip1}]")
        # This case should not be hit if points are contiguous
        raise ValueError(f"Could not find a segment for x_val = {x_val}")

    
    # 2. Get coefficients from the found segment
    i = segment_found['i']
    a_0 = segment_found['a_0 (Degree 0)']
    a_1 = segment_found['a_1 (Degree 1)']
    
    # 3. Apply the formula S_i(x) = a_1 * x + a_0
    y_val = a_1 * x_val + a_0
    
    # --- Print Intermediate Steps ---
    print("\n" + "-"*30)
    print("--- Linear Spline Evaluation ---")
    print(f"Target value x = {x_val}")
    print(f"Value is in {segment_found['Interval']} (segment i={i})")
    print(f"Using formula: S_{i}(x) = a_1 * x + a_0")
    print(f"S_{i}({x_val}) = {a_1:.12f} * {x_val} + {a_0:.12f}")
    print(f"S_{i}({x_val}) = {y_val:.12f}")
    
    return y_val

In [19]:
# 3. Evaluate the spline at a specific point (e.g., x = 1.75)
x_to_find = 1.75
y_found = evaluate_linear_spline(segments_table, x_to_find)


------------------------------
--- Linear Spline Evaluation ---
Target value x = 1.75
Value is in [1.7, 1.8] (segment i=3)
Using formula: S_3(x) = a_1 * x + a_0
S_3(1.75) = 0.813017000000 * 1.75 + 1.359132100000
S_3(1.75) = 2.781911850000
